$\newcommand{\To}{\Rightarrow}$

In [1]:
import os, sys
os.chdir('..')

In [2]:
from kernel.type import TFun, BoolType, NatType
from kernel.term import Term, Var, Eq, Implies, Inst
from kernel.thm import Thm
from data import nat
from logic import basic
from logic import matcher
from kernel import theory
from syntax.settings import settings

basic.load_theory('nat')
settings.unicode = True

## Equality

In this section, we introduce rules for working with equality. Equality is characterized by four essential properties: reflexivity, symmetry, transitivity, and congruence. They are stated as follows:

$$ \frac{}{\vdash x = x} \hbox{ reflexive} $$

$$ \frac{\vdash x = y}{\vdash y = x} \hbox{ symmetric} $$

$$ \frac{\vdash x = y \quad \vdash y = z}{\vdash x = z} \hbox{ transitive} $$

$$ \frac{\vdash f = g \quad \vdash x = y}{\vdash f\ x = g\ y} \hbox{ combination} $$

These rules can be invoked as follows:

In [3]:
x = Var("x", NatType)
y = Var("y", NatType)
z = Var("z", NatType)

print('refl x:', Thm.reflexive(x))

th1 = Thm([], Eq(x, y))
print('th1: ', th1)
print('sym(th1):', Thm.symmetric(th1))

th2 = Thm([], Eq(y, z))
print('th2: ', th2)

print('trans(th1,th2):', Thm.transitive(th1, th2))

f = Var("f", TFun(NatType, NatType))
g = Var("g", TFun(NatType, NatType))

th5 = Thm([], Eq(f, g))
print('th5:', th5)
print('comb(th5, th1):', Thm.combination(th5, th1))

refl x: ⊢ x = x
th1:  ⊢ x = y
sym(th1): ⊢ y = x
th2:  ⊢ y = z
trans(th1,th2): ⊢ x = z
th5: ⊢ f = g
comb(th5, th1): ⊢ f x = g y


These rules can be used with `implies_intr` to prove corresponding implication theorems. The following example show a variant of the combination rule, where the function stays the same.

#### Example:

Prove $x = y \to f\ x = f\ y$.

#### Solution:

0. $\vdash f = f$ by reflexive $f$.
1. $x = y \vdash x = y$ by assume $x = y$.
2. $x = y \vdash f\ x = f\ y$ by combination from 0, 1.
3. $\vdash x = y \to f\ x = f\ y$ by implies_intr $x = y$.

In Python, this is as follows:

In [4]:
th0 = Thm.reflexive(f)
th1 = Thm.assume(Eq(x, y))
th2 = Thm.combination(th0, th1)
th3 = Thm.implies_intr(Eq(x, y), th2)
print(th3)

⊢ x = y ⟶ f x = f y


We now show a more complex example, on the congruence property of functions on two variables:

#### Example:

Prove $x_1 = x_2 \to y_1 = y_2 \to h\ x_1\ y_1 = h\ x_2\ y_2$.

#### Solution:

0. $\vdash h = h$ by reflexive $h$.
1. $x_1 = x_2 \vdash x_1 = x_2$ by assume $x_1 = x_2$.
2. $y_1 = y_2 \vdash y_1 = y_2$ by assume $y_1 = y_2$.
3. $x_1 = x_2 \vdash h\ x_1 = h\ x_2$ by combination from 0, 1.
4. $x_1 = x_2, y_1 = y_2 \vdash h\ x_1\ y_1 = h\ x_2\ y_2$ by combination from 3, 2.
5. $x_1 = x_2 \vdash y_1 = y_2 \to h\ x_1\ y_1 = h\ x_2\ y_2$ by implies_intr $y_1 = y_2$ from 4.
6. $\vdash x_1 = x_2 \to y_1 = y_2 \to h\ x_1\ y_1 = h\ x_2\ y_2$ by implies_intr $x_1 = x_2$ from 5.

In [5]:
h = Var("h", TFun(NatType, NatType, NatType))
x1 = Var("x1", NatType)
x2 = Var("x2", NatType)
y1 = Var("y1", NatType)
y2 = Var("y2", NatType)
th0 = Thm.reflexive(h)
th1 = Thm.assume(Eq(x1, x2))
th2 = Thm.assume(Eq(y1, y2))
th3 = Thm.combination(th0, th1)
th4 = Thm.combination(th3, th2)
th5 = Thm.implies_intr(Eq(y1, y2), th4)
th6 = Thm.implies_intr(Eq(x1, x2), th5)
print(th6)

⊢ x1 = x2 ⟶ y1 = y2 ⟶ h x1 y1 = h x2 y2


## Equality on booleans

Two primitive deduction rules concern equality between boolean terms. `equal_intr` states that an equality between boolean terms can be proved by showing two implications. This corresponds to showing an if-and-only-if statement by showing the two directions.

$$ \frac{\vdash A \to B \quad \vdash B \to A}{\vdash A = B} \hbox{ equal_intr} $$

An example:

In [6]:
A = Var("A", BoolType)
B = Var("B", BoolType)
th0 = Thm([], Implies(A, B))
th1 = Thm([], Implies(B, A))
print(Thm.equal_intr(th0, th1))

⊢ A ⟷ B


The `equal_elim` rule states that given $A = B$ and $A$, we can obtain $B$. It shows how to make use of an equality between booleans.

$$ \frac{\vdash A = B \quad \vdash A}{\vdash B} \hbox{ equal_elim} $$

#### Example:

Prove $\vdash a = b \to P\ a \to P\ b$.

#### Solution:

0. $a = b \vdash a = b$ by assume $a = b$.
1. $P\ a \vdash P\ a$ by assume $P\ a$.
2. $\vdash P = P$ by reflexive $P$.
3. $a = b \vdash P\ a = P\ b$ by combination from 2, 0.
4. $a = b, P\ a \vdash P\ b$ by equal_elim from 3, 1.
5. $a = b \vdash P\ a \to P\ b$ by implies_intr $P\ a$.
6. $\vdash a = b \to P\ a \to P\ b$ by implies_intr $a = b$.

In [7]:
a = Var("a", NatType)
b = Var("b", NatType)
P = Var("P", TFun(NatType, BoolType))
th0 = Thm.assume(Eq(a, b))
th1 = Thm.assume(P(a))
th2 = Thm.reflexive(P)
th3 = Thm.combination(th2, th0)
th4 = Thm.equal_elim(th3, th1)
th5 = Thm.implies_intr(P(a), th4)
th6 = Thm.implies_intr(Eq(a, b), th5)
print(th6)

⊢ a = b ⟶ P a ⟶ P b


## Theorems and substitution

Once a theorem is proved, it is stored in the current theory and can be used directly in later proofs. This functionality is essential to making the length of proofs manageable. We will describe the detailed mechanism for recording theorems later. Now we will be concerned just with how to make use of existing theorems.

The theorem itself is accessed using the `get_theorem` method for theories. For example, to obtain the associativity rule for natural numbers, we use:

In [8]:
assoc_th = theory.get_theorem('add_assoc')
print(assoc_th)

⊢ ?x + ?y + ?z = ?x + (?y + ?z)


This theorem states that $(x + y) + z = x + (y + z)$ for any natural number $x,y,z$. In particular, $x,y,z$ can be replaced by arbitrary terms (of type $nat$), and the theorem still holds. To formally express this, we need another primitive deduction step for substitution. In fact, we need two primitive deduction steps: one for substitution on types, and one for substitution on terms.

The substitution rule on types is:

$$ \frac{A \vdash B}{A[\sigma] \vdash B[\sigma]} \hbox{ subst_type} $$

where $\sigma$ is an assignment of type variables. Likewise, the substitution rule on terms is:

$$ \frac{A \vdash B}{A[\sigma] \vdash B[\sigma]} \hbox{ substitution} $$

where $\sigma$ is an assignment of term variables.

We demonstrate the use of substitution rule for terms. The substitution rule for types (`subst_type`) is similar. Suppose we want to use theorem `add_assoc` to rewrite $a + b + 1$ to $a + (b + 1)$, we write:

In [9]:
eq_th = Thm.substitution(Inst(x=a, y=b, z=nat.one), assoc_th)
print(eq_th)

⊢ a + b + 1 = a + (b + 1)


Here `Inst` constructs an instantiation from a list of keyword arguments. In this example, we provided the substitution by hand. We can also use the matching procedure to get the substitutions automatically. The function below is the analog to the `rewrite` function in Section `matching`, but with theorems.

In [10]:
def rewrite_thm(th, t):
    """Rewrite t using th. Output the resulting equality."""
    inst = matcher.first_order_match(th.prop.lhs, t)
    return Thm.substitution(inst, th)

In [11]:
t = a + b + 1
print(rewrite_thm(assoc_th, t))

⊢ a + b + 1 = a + (b + 1)


We also reproduce the example from the previous section:

In [12]:
distrib_th = theory.get_theorem('distrib_l')
print(distrib_th)

t2 = a * (2 * a + (b + 1))
print(rewrite_thm(distrib_th, t2))

⊢ ?x * (?y + ?z) = ?x * ?y + ?x * ?z
⊢ a * (2 * a + (b + 1)) = a * (2 * a) + a * (b + 1)
